In [0]:
import os
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# sellers hindex
# df = pd.read_csv('hindex.csv')
# https://drive.google.com/open?id=
# downloaded = drive.CreateFile({'id':'1AOkNJt37z3qOuELDmdYCr_3E0i2cZa9P'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('train.csv') 
# uploaded = files.upload()

In [764]:
train = pd.read_csv('train_jqd04QH.csv',sep = ',')
train

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,25366,city_103,0.920,Male,Has relevent experience,Full time course,Graduate,STEM,5,<10,Pvt Ltd,1,71,0
18355,25545,city_160,0.920,Male,No relevent experience,no_enrollment,Graduate,Humanities,15,50-99,Pvt Ltd,1,160,0
18356,11514,city_114,0.926,Male,Has relevent experience,no_enrollment,Masters,STEM,11,50-99,Pvt Ltd,3,18,0
18357,1689,city_75,0.939,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10/49,Pvt Ltd,3,41,0


In [0]:
'''
Standard scaler for specific columns
'''
def stand_scaler(df,cols):
  for col in cols:
    m = df[col].mean()
    s = df[col].std()
    df[col] = df[col]-m
    df[col] = df[col]/s
  return df



In [766]:
train.groupby(['city','target']).size()

city      target
city_1    0          25
          1           2
city_10   0          86
          1           9
city_100  0         227
                   ... 
city_97   1           6
city_98   0          79
          1           7
city_99   0          90
          1          11
Length: 235, dtype: int64

In [767]:
pd.DataFrame(train.groupby(['city','gender']).size()).reset_index().sort_values(0)

,city,gender,0
0,city_1,Female,1
136,city_23,Other,1
129,city_20,Female,1
127,city_2,Female,1
126,city_19,Other,1
...,...,...,...
101,city_160,Male,595
132,city_21,Male,951
29,city_114,Male,1094
98,city_16,Male,1237


In [0]:
# train.loc[(train['city']=='city_1') & (train['gender'].isna()==True),'gender']='Male'
# train.loc[(train['city']=='city_10') & (train['gender'].isna()==True),'gender']='Male'

In [0]:
# city_function
def list_float(x):
  for i in x:
    return i

def transform_city_column(df):
  df['non_target'] = 1-df['target']
  city_analysis = df.groupby(['city']).agg('mean')[['target','non_target']]
  city_dict = city_analysis.iloc[:,:1].T.to_dict('list')
  df['city'] = df['city'].map(city_dict).map(list_float)
  df = df.drop(['non_target'],axis=1)
  return df, city_dict

def transform_enrolled_university_column(df):
  df['non_target'] = 1-df['target']
  city_analysis = df.groupby(['city']).agg('mean')[['target','non_target']]
  city_dict = city_analysis.iloc[:,:1].T.to_dict('list')
  df['city'] = df['city'].map(city_dict).map(list_float)
  df = df.drop(['non_target'],axis=1)
  return df, city_dict


In [0]:
train,city_map = transform_city_column(train)

In [0]:
# train[train['training_hours']=='NGO']#,'city'].drop_duplicates() #.groupby('enrolled_university').size()#,'city_development_index'].describe()
train.loc[(train['gender'].isna()==True) & (train['training_hours']<=2),'gender']='Male'
# train.loc[(train['city_development_index']<0.49) & (train['gender'].isna()==True),'gender']='Male'

In [772]:
# train[train['enrollee_id'].isna()==True]
# train[train['city'].isna()==True] 123 unique values
# train[train['city_development_index'].isna()==True] 90 unique values
'''
train[train['gender'].isna()==True] #4000 empty rows for gender 
so we won't be using gender as of now
'''

# train[train['relevent_experience'].isna()==True]

'''
train[train['enrolled_university'].isna()==True] #342 missing rows
'''

# train[train['education_level'].isna()==True] #457 missing rows
train


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,0.123810,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,0.098592,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,0.093108,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,0.100840,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,0.162362,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,25366,0.129188,0.920,Male,Has relevent experience,Full time course,Graduate,STEM,5,<10,Pvt Ltd,1,71,0
18355,25545,0.118501,0.920,Male,No relevent experience,no_enrollment,Graduate,Humanities,15,50-99,Pvt Ltd,1,160,0
18356,11514,0.092391,0.926,Male,Has relevent experience,no_enrollment,Masters,STEM,11,50-99,Pvt Ltd,3,18,0
18357,1689,0.096386,0.939,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10/49,Pvt Ltd,3,41,0


In [0]:
# train  = train[(train['company_type'].isna()==False)]# &(train['company_size'].isna()==True) ]




In [0]:
# Final var list - city_development_index,relevent_experience

In [774]:
import numpy as np
df = train.drop(['enrollee_id'],axis=1)

# df['city'] = df['city'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['city'])
df.loc[(df['gender'].isna()==True) & (df['training_hours']<=2),'gender']='Male'
df.loc[(df['city_development_index']<0.47) & (df['gender'].isna()==True),'gender']='Male'

df['gender'] = df['gender'].fillna('Unknown')
df = pd.get_dummies(df,columns=['gender'])
# df = df.drop(['city_development_index'],axis=1)
# df = pd.get_dummies(df,columns=['relevent_experience'])

df['rel_exp']=None
df.loc[df['relevent_experience']=='Has relevent experience','rel_exp']=1
df.loc[df['relevent_experience']=='No relevent experience','rel_exp']=0
df = df.drop(['relevent_experience'],axis=1)

# df['rel_exp']=None
# df.loc[df['relevent_experience']=='Has relevent experience','rel_exp']=0
# df.loc[df['relevent_experience']=='No relevent experience','rel_exp']=1
# df = df.drop(['relevent_experience'],axis=1)

df['enrolled_university'] = df['enrolled_university'].fillna('Unknown')
df = pd.get_dummies(df,columns=['enrolled_university'])

# df['college']=None
# df.loc[df['enrolled_university']=='no_enrollment','college']=0
# df.loc[df['enrolled_university'].isna()==True,'college']=0
# df.loc[df['enrolled_university']=='Full time course','college']=2
# df.loc[df['enrolled_university']=='Part time course','college']=1
# df = df.drop(['enrolled_university'],axis=1)



# df['education_level'] = df['education_level'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['education_level'])


df['edu_level']=None
df.loc[df['education_level']=='High School','edu_level']=2
df.loc[df['education_level'].isna()==True,'edu_level']=0
df.loc[df['education_level']=='Masters','edu_level']=4
df.loc[df['education_level']=='Graduate','edu_level']=3
df.loc[df['education_level']=='Phd','edu_level']=5
df.loc[df['education_level']=='Primary School','edu_level']=1
df = df.drop(['education_level'],axis=1)


# df['major_discipline'] = df['major_discipline'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['major_discipline'])
df = df.drop(['major_discipline'],axis=1)


df['experience'] = df['experience'].fillna(1)
df['experience'] = df['experience'].str.replace('<1','0')
df['experience'] = df['experience'].str.replace('>','')
df['experience'] = df['experience'].fillna(0)
df['experience'] = pd.to_numeric(df['experience'])
df['experience'] = df['experience'].fillna(0)

df['company_type'] = df['company_type'].fillna('Unknown')
df = pd.get_dummies(df,columns=['company_type'])

# df = df.drop(['company_type'],axis=1)
# df['company_size'] = df['company_size'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['company_size'])
# df['cmpny_size']=None
# df.loc[df['company_size'].isna()==True,'cmpny_size']=4
# df.loc[df['company_size']=='100-500','cmpny_size']=4
# df.loc[df['company_size']=='<10','cmpny_size']=1
# df.loc[df['company_size']=='50-99','cmpny_size']=3
# df.loc[df['company_size']=='5000-9999','cmpny_size']=7
# df.loc[df['company_size']=='10000+','cmpny_size']=8
# df.loc[df['company_size']=='1000-4999','cmpny_size']=6
# df.loc[df['company_size']=='500-999','cmpny_size']=5
# df.loc[df['company_size']=='10/49','cmpny_size']=2
df = df.drop(['company_size'],axis=1)



df.loc[df['last_new_job'].isna()==True,'last_new_job'] =0 #df.loc[df['last_new_job'].isna()==True,'experience']
df.loc[df['last_new_job']=='never','last_new_job'] = 6#df.loc[df['last_new_job']=='never','experience']
df['last_new_job'] = df['last_new_job'].str.replace('>4','4')
df['last_new_job'] = pd.to_numeric(df['last_new_job'])
df.loc[df['last_new_job'].isna()==True,'last_new_job'] = 0#df.loc[df['last_new_job'].isna()==True,'experience']
# df['last_new_job'].drop_duplicates()

df

,city,city_development_index,experience,last_new_job,training_hours,target,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,0.123810,0.689,3,1.0,106,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
1,0.098592,0.923,14,1.0,69,0,0,1,0,0,1,0,0,0,1,3,0,1,0,0,0,0,0
2,0.093108,0.910,6,2.0,4,0,0,0,0,1,1,0,0,0,1,3,0,0,0,0,1,0,0
3,0.100840,0.666,14,1.0,26,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
4,0.162362,0.887,8,2.0,88,1,0,0,0,1,0,0,0,0,1,4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,0.129188,0.920,5,1.0,71,0,0,1,0,0,1,1,0,0,0,3,0,0,0,0,0,1,0
18355,0.118501,0.920,15,1.0,160,0,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0
18356,0.092391,0.926,11,3.0,18,0,0,1,0,0,1,0,0,0,1,4,0,0,0,0,0,1,0
18357,0.096386,0.939,20,3.0,41,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0


In [775]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


avg = df['training_hours'].mean()

df['training_hours'] = (df['training_hours'])/avg
df


avg_experience = df['experience'].mean()
df['experience'] = (df['experience'])/avg_experience

# avg_last_new_job = df['last_new_job'].mean()
# df['last_new_job'] = (df['last_new_job'])/avg_last_new_job

df

# df = stand_scaler(df,['training_hours','experience','city','city_development_index'])
df

,city,city_development_index,experience,last_new_job,training_hours,target,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,0.123810,0.689,0.289525,1.0,1.608522,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
1,0.098592,0.923,1.351119,1.0,1.047057,0,0,1,0,0,1,0,0,0,1,3,0,1,0,0,0,0,0
2,0.093108,0.910,0.579051,2.0,0.060699,0,0,0,0,1,1,0,0,0,1,3,0,0,0,0,1,0,0
3,0.100840,0.666,1.351119,1.0,0.394543,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
4,0.162362,0.887,0.772068,2.0,1.335377,1,0,0,0,1,0,0,0,0,1,4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,0.129188,0.920,0.482542,1.0,1.077406,0,0,1,0,0,1,1,0,0,0,3,0,0,0,0,0,1,0
18355,0.118501,0.920,1.447627,1.0,2.427957,0,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0
18356,0.092391,0.926,1.061593,3.0,0.273145,0,0,1,0,0,1,0,0,0,1,4,0,0,0,0,0,1,0
18357,0.096386,0.939,1.930169,3.0,0.622164,0,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0


In [776]:
df.columns

Index(['city', 'city_development_index', 'experience', 'last_new_job',
       'training_hours', 'target', 'gender_Female', 'gender_Male',
       'gender_Other', 'gender_Unknown', 'rel_exp',
       'enrolled_university_Full time course',
       'enrolled_university_Part time course', 'enrolled_university_Unknown',
       'enrolled_university_no_enrollment', 'edu_level',
       'company_type_Early Stage Startup', 'company_type_Funded Startup',
       'company_type_NGO', 'company_type_Other', 'company_type_Public Sector',
       'company_type_Pvt Ltd', 'company_type_Unknown'],
      dtype='object')

In [777]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report 
from sklearn.metrics import precision_score,recall_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

x= df.drop(['target'],axis=1)
y= df['target']
print(y.value_counts())
# tree = DecisionTreeClassifier(criterion = "entropy", random_state = 1000,max_depth=6, min_samples_leaf=5).fit(x, y)  # 6,5

# cv_score = cross_val_score(tree,x,y,cv=10,scoring= 'accuracy',n_jobs = -1)
# print(cv_score.mean())
# tree.fit(x, y)
# predicted = tree.predict(x)

# svc = SVC(C=50,kernel='poly',degree=7,class_weight='balanced',random_state=0).fit(x, y)
# predicted = svc.predict(x)

# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))
# print(roc_auc_score(y, predicted))
# reg = LinearRegression().fit(x, y)
# predicted = reg.predict(x)
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))


# clf = LogisticRegression(random_state=0).fit(x, y)
# predicted = clf.predict(x)
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='gini', n_estimators = 20,random_state=0,max_depth=5, min_samples_leaf=6,class_weight='balanced')
rf.fit(x, y)  
# predicted = rf.predict(x)
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))

# svm = SVC(gamma='auto').fit(x, y)
# predicted = svm.predict(x)
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))




0    15934
1     2425
Name: target, dtype: int64


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:

x= df.drop(['target'],axis=1)
y= df['target']

# x = pd.DataFrame(scaler.fit_transform(x),index=x.index, columns=x.columns)
# tree = DecisionTreeClassifier(criterion = "entropy", 
#             random_state = 1000,max_depth=1000000, min_samples_leaf=2) 
# tree.fit(x, y)
# x['tree_output'] = tree.predict(x)

predicted_tree = rf.predict(x)
# predicted_svc = svc.predict(x)

# x['predicted_tree'], x['predicted_svc'] = predicted_tree, predicted_svc
x['predicted_tree'] = predicted_tree

from sklearn.metrics import r2_score
# from sklearn.linear_model import Lasso
# r2_score(y, predicted)

from sklearn.linear_model import Ridge
clf = Ridge(alpha=65,fit_intercept=False,solver='sparse_cg').fit(x, y) #65

predicted = clf.predict(x)
# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y, predicted))

In [779]:
x.columns

Index(['city', 'city_development_index', 'experience', 'last_new_job',
       'training_hours', 'gender_Female', 'gender_Male', 'gender_Other',
       'gender_Unknown', 'rel_exp', 'enrolled_university_Full time course',
       'enrolled_university_Part time course', 'enrolled_university_Unknown',
       'enrolled_university_no_enrollment', 'edu_level',
       'company_type_Early Stage Startup', 'company_type_Funded Startup',
       'company_type_NGO', 'company_type_Other', 'company_type_Public Sector',
       'company_type_Pvt Ltd', 'company_type_Unknown', 'predicted_tree'],
      dtype='object')

In [780]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y, predicted)

0.6760739985429588

In [0]:

# downloaded = drive.CreateFile({'id':'1dAwY82FjplXkYrrkrzJdfTJzEQ9SVASy'}) # replace the id with id of file you want to access
# downloaded.GetContentFile('test.csv') 
# test = pd.read_csv('test.csv',sep=',')
# test
# uploaded = files.upload()

In [782]:
test = pd.read_csv('test_KaymcHn.csv',sep=',')
test

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,16548,city_33,0.448,NaN,No relevent experience,Full time course,Graduate,STEM,<1,1000-4999,Public Sector,NaN,15
1,12036,city_28,0.939,Male,No relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1,94
2,11061,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,3,NaN,NaN,1,17
3,5032,city_104,0.924,Male,No relevent experience,no_enrollment,Phd,STEM,>20,50-99,Pvt Ltd,2,76
4,17599,city_77,0.830,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,<10,Pvt Ltd,2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15016,11308,city_46,0.762,NaN,Has relevent experience,no_enrollment,Masters,STEM,>20,500-999,Pvt Ltd,>4,68
15017,14612,city_21,0.624,Male,Has relevent experience,Full time course,Masters,STEM,4,1000-4999,Pvt Ltd,1,320
15018,33346,city_16,0.910,Male,Has relevent experience,no_enrollment,High School,NaN,9,1000-4999,Pvt Ltd,4,13
15019,14506,city_64,0.666,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,1,38


In [783]:
# city_function
def list_float_test(x):
  for i in x:
    return i

def transform_city_column(df,city_map):
  df['city'] = df['city'].map(city_map).map(list_float_test)
  return df

df = test#.drop(['city'],axis=1)
df = transform_city_column(df,city_map)
# df.loc[(df['city']=='city_1') & (df['gender'].isna()==True),'gender']='Male'
# df.loc[(df['city']=='city_10') & (df['gender'].isna()==True),'gender']='Male'

# df['city'] = df['city'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['city'])
df.loc[(df['gender'].isna()==True) & (df['training_hours']<=2),'gender']='Male'
df.loc[(df['city_development_index']<0.47) & (df['gender'].isna()==True),'gender']='Male'
df['gender'] = df['gender'].fillna('Unknown')
df = pd.get_dummies(df,columns=['gender'])
# df = df.drop(['city_development_index'],axis=1)
# df = pd.get_dummies(df,columns=['relevent_experience'])

df['rel_exp']=None
df.loc[df['relevent_experience']=='Has relevent experience','rel_exp']=1
df.loc[df['relevent_experience']=='No relevent experience','rel_exp']=0
df = df.drop(['relevent_experience'],axis=1)

# df['rel_exp']=None
# df.loc[df['relevent_experience']=='Has relevent experience','rel_exp']=0
# df.loc[df['relevent_experience']=='No relevent experience','rel_exp']=1
# df = df.drop(['relevent_experience'],axis=1)

df['enrolled_university'] = df['enrolled_university'].fillna('Unknown')
df = pd.get_dummies(df,columns=['enrolled_university'])

# df['college']=None
# df.loc[df['enrolled_university']=='no_enrollment','college']=0
# df.loc[df['enrolled_university'].isna()==True,'college']=0
# df.loc[df['enrolled_university']=='Full time course','college']=2
# df.loc[df['enrolled_university']=='Part time course','college']=1
# df = df.drop(['enrolled_university'],axis=1)


# df['education_level'] = df['education_level'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['education_level'])


df['edu_level']=None
df.loc[df['education_level']=='High School','edu_level']=2
df.loc[df['education_level'].isna()==True,'edu_level']=0
df.loc[df['education_level']=='Masters','edu_level']=4
df.loc[df['education_level']=='Graduate','edu_level']=3
df.loc[df['education_level']=='Phd','edu_level']=5
df.loc[df['education_level']=='Primary School','edu_level']=1
df = df.drop(['education_level'],axis=1)


# df['edu_level']=None
# df.loc[df['education_level']=='High School','edu_level']=3
# df.loc[df['education_level'].isna()==True,'edu_level']=0
# df.loc[df['education_level']=='Masters','edu_level']=4
# df.loc[df['education_level']=='Graduate','edu_level']=5
# df.loc[df['education_level']=='Phd','edu_level']=2
# df.loc[df['education_level']=='Primary School','edu_level']=1
# df = df.drop(['education_level'],axis=1)

# df['major_discipline'] = df['major_discipline'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['major_discipline'])
df = df.drop(['major_discipline'],axis=1)

df['experience'] = df['experience'].fillna(1)
df['experience'] = df['experience'].str.replace('<1','0')
df['experience'] = df['experience'].str.replace('>','')
df['experience'] = df['experience'].fillna(0)
df['experience'] = pd.to_numeric(df['experience'])
df['experience'] = df['experience'].fillna(0)

df['company_type'] = df['company_type'].fillna('Unknown')
df = pd.get_dummies(df,columns=['company_type'])

# df = df.drop(['company_type'],axis=1)
# df['company_size'] = df['company_size'].fillna('Unknown')
# df = pd.get_dummies(df,columns=['company_size'])
# df['cmpny_size']=None
# df.loc[df['company_size'].isna()==True,'cmpny_size']=4
# df.loc[df['company_size']=='100-500','cmpny_size']=4
# df.loc[df['company_size']=='<10','cmpny_size']=1
# df.loc[df['company_size']=='50-99','cmpny_size']=3
# df.loc[df['company_size']=='5000-9999','cmpny_size']=7
# df.loc[df['company_size']=='10000+','cmpny_size']=8
# df.loc[df['company_size']=='1000-4999','cmpny_size']=6
# df.loc[df['company_size']=='500-999','cmpny_size']=5
# df.loc[df['company_size']=='10/49','cmpny_size']=2
df = df.drop(['company_size'],axis=1)


df.loc[df['last_new_job'].isna()==True,'last_new_job'] =0 #df.loc[df['last_new_job'].isna()==True,'experience']
df.loc[df['last_new_job']=='never','last_new_job'] = 6#df.loc[df['last_new_job']=='never','experience']
df['last_new_job'] = df['last_new_job'].str.replace('>4','4')
df['last_new_job'] = pd.to_numeric(df['last_new_job'])
df.loc[df['last_new_job'].isna()==True,'last_new_job'] = 0#df.loc[df['last_new_job'].isna()==True,'experience']
# df['last_new_job'].drop_duplicates()


# df['last_new_job'] = df['last_new_job'].fillna(0)
# df['last_new_job'] = df['last_new_job'].str.replace('>4','6')
# df['last_new_job'] = df['last_new_job'].str.replace('never','0')
# df['last_new_job'] = df['last_new_job'].str.replace('never','0')
# df['last_new_job'] = pd.to_numeric(df['last_new_job'])
# df['last_new_job'] = df['last_new_job'].fillna(0)
# df['last_new_job'].drop_duplicates()

df

,enrollee_id,city,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,16548,0.538462,0.448,0,0.0,15,0,1,0,0,0,1,0,0,0,3,0,0,0,0,1,0,0
1,12036,0.075472,0.939,20,1.0,94,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0
2,11061,0.129188,0.920,3,1.0,17,0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,1
3,5032,0.066038,0.924,20,2.0,76,0,1,0,0,0,0,0,0,1,5,0,0,0,0,0,1,0
4,17599,0.026316,0.830,6,2.0,65,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15016,11308,0.164062,0.762,20,4.0,68,0,0,0,1,1,0,0,0,1,4,0,0,0,0,0,1,0
15017,14612,0.270933,0.624,4,1.0,320,0,1,0,0,1,1,0,0,0,4,0,0,0,0,0,1,0
15018,33346,0.093108,0.910,9,4.0,13,0,1,0,0,1,0,0,0,1,2,0,0,0,0,0,1,0
15019,14506,0.100840,0.666,5,1.0,38,0,0,0,1,0,1,0,0,0,3,0,0,0,0,0,0,1


In [784]:
from sklearn.preprocessing import MinMaxScaler

# df['training_hours'] = np.log(df['training_hours'])#/avg

df['training_hours'] = df['training_hours']/df['training_hours'].mean() #avg
df

df['experience'] = (df['experience'])/df['experience'].mean() #avg_experience
df

# df['last_new_job'] = (df['last_new_job'])/avg_last_new_job
# df

# df = stand_scaler(df,['training_hours','experience','city','city_development_index'])
# df


,enrollee_id,city,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,16548,0.538462,0.448,0.000000,0.0,0.230209,0,1,0,0,0,1,0,0,0,3,0,0,0,0,1,0,0
1,12036,0.075472,0.939,1.936257,1.0,1.442643,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0
2,11061,0.129188,0.920,0.290439,1.0,0.260904,0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,1
3,5032,0.066038,0.924,1.936257,2.0,1.166392,0,1,0,0,0,0,0,0,1,5,0,0,0,0,0,1,0
4,17599,0.026316,0.830,0.580877,2.0,0.997572,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15016,11308,0.164062,0.762,1.936257,4.0,1.043614,0,0,0,1,1,0,0,0,1,4,0,0,0,0,0,1,0
15017,14612,0.270933,0.624,0.387251,1.0,4.911126,0,1,0,0,1,1,0,0,0,4,0,0,0,0,0,1,0
15018,33346,0.093108,0.910,0.871316,4.0,0.199514,0,1,0,0,1,0,0,0,1,2,0,0,0,0,0,1,0
15019,14506,0.100840,0.666,0.484064,1.0,0.583196,0,0,0,1,0,1,0,0,0,3,0,0,0,0,0,0,1


In [785]:
df

,enrollee_id,city,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown
0,16548,0.538462,0.448,0.000000,0.0,0.230209,0,1,0,0,0,1,0,0,0,3,0,0,0,0,1,0,0
1,12036,0.075472,0.939,1.936257,1.0,1.442643,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0
2,11061,0.129188,0.920,0.290439,1.0,0.260904,0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,1
3,5032,0.066038,0.924,1.936257,2.0,1.166392,0,1,0,0,0,0,0,0,1,5,0,0,0,0,0,1,0
4,17599,0.026316,0.830,0.580877,2.0,0.997572,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15016,11308,0.164062,0.762,1.936257,4.0,1.043614,0,0,0,1,1,0,0,0,1,4,0,0,0,0,0,1,0
15017,14612,0.270933,0.624,0.387251,1.0,4.911126,0,1,0,0,1,1,0,0,0,4,0,0,0,0,0,1,0
15018,33346,0.093108,0.910,0.871316,4.0,0.199514,0,1,0,0,1,0,0,0,1,2,0,0,0,0,0,1,0
15019,14506,0.100840,0.666,0.484064,1.0,0.583196,0,0,0,1,0,1,0,0,0,3,0,0,0,0,0,0,1


In [786]:

# 'city', 'city_development_index', 'experience', 'last_new_job',
#        'training_hours', 'non_target', 'gender_Female', 'gender_Male',
#        'gender_Other', 'gender_Unknown', 'rel_exp',
#        'enrolled_university_Full time course',
#        'enrolled_university_Part time course', 'enrolled_university_Unknown',
#        'enrolled_university_no_enrollment', 'edu_level',
#        'company_type_Early Stage Startup', 'company_type_Funded Startup',
#        'company_type_NGO', 'company_type_Other', 'company_type_Public Sector',
#        'company_type_Pvt Ltd', 'company_type_Unknown', 'predicted'


x= df.drop(['enrollee_id'],axis=1)
# x = pd.DataFrame(scaler.fit_transform(x),index=x.index, columns=x.columns)

# predicted_tree,predicted_svc = tree.predict(x),svc.predict(x)
predicted_tree = rf.predict(x)
# x['predicted_tree'], x['predicted_svc'] = predicted_tree,predicted_svc

x['predicted_tree'] = predicted_tree
df['target'] = clf.predict(x)
df



,enrollee_id,city,city_development_index,experience,last_new_job,training_hours,gender_Female,gender_Male,gender_Other,gender_Unknown,rel_exp,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_Unknown,enrolled_university_no_enrollment,edu_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,company_type_Unknown,target
0,16548,0.538462,0.448,0.000000,0.0,0.230209,0,1,0,0,0,1,0,0,0,3,0,0,0,0,1,0,0,0.307130
1,12036,0.075472,0.939,1.936257,1.0,1.442643,0,1,0,0,0,0,0,0,1,3,0,0,0,0,0,1,0,0.053450
2,11061,0.129188,0.920,0.290439,1.0,0.260904,0,1,0,0,0,1,0,0,0,3,0,0,0,0,0,0,1,0.234289
3,5032,0.066038,0.924,1.936257,2.0,1.166392,0,1,0,0,0,0,0,0,1,5,0,0,0,0,0,1,0,0.079184
4,17599,0.026316,0.830,0.580877,2.0,0.997572,0,1,0,0,1,0,0,0,1,3,0,0,0,0,0,1,0,0.070429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15016,11308,0.164062,0.762,1.936257,4.0,1.043614,0,0,0,1,1,0,0,0,1,4,0,0,0,0,0,1,0,0.218282
15017,14612,0.270933,0.624,0.387251,1.0,4.911126,0,1,0,0,1,1,0,0,0,4,0,0,0,0,0,1,0,0.244237
15018,33346,0.093108,0.910,0.871316,4.0,0.199514,0,1,0,0,1,0,0,0,1,2,0,0,0,0,0,1,0,0.081358
15019,14506,0.100840,0.666,0.484064,1.0,0.583196,0,0,0,1,0,1,0,0,0,3,0,0,0,0,0,0,1,0.233417


In [0]:
df[['enrollee_id','target']].to_csv('final_sub.csv',index=False)

In [0]:
# max(reg.coef_)